In [1]:
import numpy as np
import tensorflow as tf

import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import sys
sys.path.append(r'D:\Users\Madhouse\tensorflow\yobiface\src')
import utils

import os
from os.path import join as pjoin
import sys
import copy
import shutil
import pickle

print(tf.__version__)

d:\Users\Madhouse\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.7.0


In [2]:
def load_all_pics(path):
    data = {}
    pics_ctr = 0
    files= os.listdir(path)
    for file in files: #遍历文件夹 
        if not os.path.isdir(file): #判断是否是文件夹，不是文件夹才打开
        #print(file)
            person_dir = pjoin(path, file)
            if(file.find(".jpg") > 0):
                guy=file.split("/")[-1][:-4]
                #print("guy[%d]=%s"%(pics_ctr,guy))
                curr_pics = [utils.preproc(cv2.imread(person_dir))]
                data[guy] = curr_pics
                pics_ctr += len(curr_pics)
    return data, pics_ctr

In [3]:
tf.reset_default_graph()
sess = tf.Session()
saver = tf.train.import_meta_graph(r'D:\Users\Madhouse\tensorflow\download\20180402-114759\model-20180402-114759.meta')
saver.restore(sess, r'D:\Users\Madhouse\tensorflow\download\20180402-114759\model-20180402-114759.ckpt-275')
images_placeholder = tf.get_default_graph().get_tensor_by_name("input:0")
embeddings = tf.get_default_graph().get_tensor_by_name("embeddings:0")
phase_train_placeholder = tf.get_default_graph().get_tensor_by_name("phase_train:0")

INFO:tensorflow:Restoring parameters from D:\Users\Madhouse\tensorflow\download\20180402-114759\model-20180402-114759.ckpt-275


In [31]:
DATA_DIR = r'D:\software\myphotobase\yue'
data, n_pics = load_all_pics(DATA_DIR)
print('Loaded %d pics.' % n_pics)

test_dict = {}
for p in data:
    test_dict[p] = list(sess.run([embeddings], feed_dict={images_placeholder: np.array(data[p]), phase_train_placeholder: False })[0])

print('Images:')
for g in test_dict.keys():
    print(' %s' % g)
    #print(test_dict[g])
print('')

#print(test_dict)

# Print distance matrix
print('Distance matrix')
print('      ', end='')
for i in test_dict.keys():
    print('     %s ' % i, end='')
print('')
for i in test_dict.keys():
    print('%s  ' % i, end='')
    for j in test_dict.keys():
        #dist = np.sqrt(np.sum(np.square(np.subtract(emb[i,:], emb[j,:]))))
        dist = np.linalg.norm(np.array(test_dict[i]) - np.array(test_dict[j]))
        print('    %1.4f  ' % dist, end='')
    print('')

Loaded 3 pics.
Images:
 yue_1
 yue_2
 yue_3

Distance matrix
           yue_1      yue_2      yue_3 
yue_1      0.0000      0.8892      0.6402  
yue_2      0.8892      0.0000      0.9954  
yue_3      0.6402      0.9954      0.0000  
